In [33]:
import cv2
import numpy as np

In [34]:
def find_text_contours(image):
    """
    이미지에서 글씨 윤곽선을 찾아 반환합니다.
    
    Args:
        image: 입력 이미지 (BGR 채널)
    
    Returns:
        글씨 윤곽선 리스트
    """

    # 이미지를 그레이스케일로 변환합니다.
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    
    # 가우시안 블러 적용으로 노이즈 제거
    gray = cv2.GaussianBlur(gray, (5, 5), 0)
    
    # Canny 에지 검출 알고리즘 적용
    edges = cv2.Canny(gray, 100, 100)
    _, thresh = cv2.threshold(gray, 127, 255, cv2.THRESH_BINARY)
    adaptive_threshold= cv2.adaptiveThreshold(gray, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY, 11, 2)
    
    thresh = cv2.erode(thresh, None, iterations=2)
    thresh = cv2.dilate(thresh, None, iterations=2)
    
    # 윤곽선 추출
    contours, _ = cv2.findContours(edges, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
    return contours

In [35]:
def group_nearby_contours(contours):
    """
    가까운 윤곽선들을 그룹으로 묶습니다.
    
    Args:
        contours: 글씨 윤곽선 리스트
    
    Returns:
        그룹화된 윤곽선 리스트
    """
    
    # 최소 외곽 사각형 계산
    bounding_boxes = [cv2.boundingRect(contour) for contour in contours]
    
    # Hungarian 알고리즘을 이용한 최적의 매칭 찾기
    cost_matrix = np.zeros((len(bounding_boxes), len(bounding_boxes)))
    for i, bbox1 in enumerate(bounding_boxes):
        for j, bbox2 in enumerate(bounding_boxes):
            # 두 박스 중심 거리 계산
            cost_matrix[i, j] = np.linalg.norm(np.array(bbox1[0:2]) - np.array(bbox2[0:2]))

    # 최적의 매칭 결과 얻기
    matched_pairs = cv2.minMaxLoc(cost_matrix)[2]
    
    # 그룹화된 윤곽선 리스트 생성
    grouped_contours = []
    for i in range(len(matched_pairs)):
        if matched_pairs[i] != i:
            continue
    
        group = [contours[i]]
        for j in range(len(matched_pairs)):
            if matched_pairs[j] == i:
                group.append(contours[j])
    
        grouped_contours.append(group)
    
    return grouped_contours

In [36]:
def mark_text_regions(image, grouped_contours):
    """
    이미지에 글씨 영역을 사각형으로 표시합니다.
    
    Args:
        image: 입력 이미지 (BGR 채널)
        grouped_contours: 그룹화된 윤곽선 리스트
    
    Returns:
        글씨 영역이 표시된 이미지
    """
    
    for group in grouped_contours:
        # 전체 그룹의 최소 외곽 사각형 계산
        x, y, w, h = cv2.boundingRect(np.vstack(group))
        
        # 사각형 표시
        cv2.rectangle(image, (x, y), (x+w, y+h), (0, 255, 0), 2)
        
    return image

In [37]:
# 이미지 불러오기
image = cv2.imread("../dataset_dir/13.한국어글자체/01.손글씨/01_handwriting_sentence_images/1_sentence/00000002.png")

# 글씨 윤곽선 찾기
contours = find_text_contours(image)

# 가까운 윤곽선 그룹화
grouped_contours = group_nearby_contours(contours)

# 글씨 영역 표시
marked_image = mark_text_regions(image, grouped_contours)

In [38]:
from time import time

# 결과 이미지 출력
cv2.imwrite(f'./image/{time()}.jpg', image)

True